# Data pre-processing

In [2]:
import torch
import torchaudio
import sys
import os
sys.path.append(os.path.abspath('../src'))
from segmentation import segment_cough
import datasets
import glob
import numpy as np

import cv2
SAMPLE_RATE = 16000

In [3]:
data_folder = "../valid_data/"

In [4]:
import glob, os
os.chdir(data_folder)
sound_files =  sorted([f for f in glob.glob("*.wav")])


In [5]:
metadata = datasets.load_dataset("csv", os.path.join(data_folder, "metadata_compiled_valid.csv"))
metadata = metadata['validation']

Using custom data configuration ../valid_data/metadata_compiled_valid.csv-24bdde2a7bd56514
Found cached dataset csv (/Users/nantanick/.cache/huggingface/datasets/csv/../valid_data/metadata_compiled_valid.csv-24bdde2a7bd56514/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
assert len(sound_files) == len(metadata)

In [7]:
#small version
sound_files= sound_files[:20]
metadata = metadata.select(range(20))

In [8]:
audio = datasets.Dataset.from_dict(
    {
        "audio_path": sound_files,
    }
)

In [13]:
def load_audio(x):
    tensor, sample_rate = torchaudio.load(x['audio_path'])
    tensor = torchaudio.functional.resample(tensor, sample_rate, SAMPLE_RATE)
    return {'audio':tensor}

audio = audio.map(load_audio).with_format("numpy")

  0%|          | 0/20 [00:00<?, ?ex/s]

In [ ]:
# def segment_audio(example):
#     """
#     Takes the first 3 coughs (max 1 sec)
#     and put it in an array
#     """
#     x = example['audio'][0]
#     coughs, _ = segment_cough(x, SAMPLE_RATE)
#     arr = coughs[0]
#     res = np.zeros(SAMPLE_RATE)
#     _min = min(len(arr), SAMPLE_RATE)
#     res[:_min] = arr[:_min]
#     return {'coughs':res}

# audio = audio.map(segment_audio)

In [ ]:
SAMPLE_RATE = 16000
NUM_SAMPLES = SAMPLE_RATE*10

def _cut_if_necessary(signal):
    if signal.shape[1] > self.num_samples:
        signal = signal[:, :self.num_samples]
    return signal

def _right_pad_if_necessary(signal):
    length_signal = signal.shape[1]
    if length_signal < NUM_SAMPLES:
        num_missing_samples = NUM_SAMPLES - length_signal
        last_dim_padding = (0, num_missing_samples)
        signal = torch.nn.functional.pad(signal, last_dim_padding)
    return signal

def _resample_if_necessary(signal, sr):
    if sr != SAMPLE_RATE:
        resampler = torchaudio.transforms.Resample(sr, SAMPLE_RATE)
        signal = resampler(signal)
    return signal

def _mix_down_if_necessary(self, signal):
    if signal.shape[0] > 1:
        signal = torch.mean(signal, dim=0, keepdim=True)
    return signal

def segment_audio(example):
    signal = example['audio']
    signal = _resample_if_necessary(signal, sr)
    signal = _mix_down_if_necessary(signal)
    signal = _cut_if_necessary(signal)
    signal = _right_pad_if_necessary(signal)
    signal = transformation(signal)
    


In [121]:
# dataset = metadata.add_column('audio', audio['audio'])
dataset = datasets.concatenate_datasets([audio, metadata], axis=1)

In [122]:
np.array(dataset['coughs'][0]).shape

(3, 16000)

In [123]:
keeps = ['audio', 'status', 'status_SSL']
remove = [c for c in dataset.features if c not in keeps]
ds_torch = dataset.remove_columns(remove).with_format("torch")

In [125]:
tmp = ds_torch['audio'][0]

In [109]:
X = ds_torch['coughs']
y = ds_torch['status']

In [ ]:
class MyPipeline(torch.nn.Module):
    def __init__(
        self,
        n_fft=1024,
        n_mel=256,
        stretch_factor=0.8,
        sample_rate = 16000
    ):
        super().__init__()
        
        self.spec = Spectrogram(n_fft=n_fft, power=2)
        self.sample_rate= sample_rate
        

#         self.spec_aug = torch.nn.Sequential(
#             TimeStretch(stretch_factor, fixed_rate=True),
#             FrequencyMasking(freq_mask_param=80),
#             TimeMasking(time_mask_param=80),
#         )

        self.mel_scale = MelScale(
            n_mels=n_mel, sample_rate=resample_freq, n_stft=n_fft // 2 + 1)

    def forward(self, waveform: torch.Tensor) -> torch.Tensor:
        
        wavefrom = torchaudio.transforms.PitchShift(waveform, self.sample_rate) 
        # Convert to power spectrogram
        spec = self.spec(waveform)

#         # Apply SpecAugment
#         spec = self.spec_aug(spec)

        # Convert to mel-scale
        mel = self.mel_scale(spec)
        cv2.resize(np.array(img), (256,256)).shape

        return mel

In [116]:
spec =  torchaudio.transforms.Spectrogram(n_fft=1024, power=2)

In [132]:
img = np.squeeze(spec(tmp))

In [133]:
cv2.resize(np.array(img), (256,256)).shape

(256, 256)

In [131]:
img.shape

torch.Size([1, 513, 306])